In [13]:
import pandas as pd
import csv

In [2]:
trunc_hiseq_rna_df = pd.read_table('data/trunc_illuminahiseq_rna_data.tsv', header=0, skiprows=[1], index_col=0)
trunc_ga_rna_df = pd.read_table('data/trunc_illuminaga_rna_data.tsv', header=0, skiprows=[1], index_col=0)
clinical_df = pd.read_table('data/clinical/COADREAD.clin.merged.txt', index_col=0)

### Get a set of patients that we know is in the clinical data

In [3]:
clinical_patient_list = []
column_header_list = list(clinical_df.columns.values)
for column_header in column_header_list:
    clinical_patient_list.append(clinical_df.loc["patient.bcr_patient_barcode"][column_header].upper()) 
    
clinical_patient_set = set(clinical_patient_list)

In [4]:
hiseq_patient_set = set(list(trunc_hiseq_rna_df.columns.values))
ga_patient_set = set(list(trunc_ga_rna_df.columns.values))
len(ga_patient_set)

263

#### NOTE: We're defaulting to GA RNA patient data over Hiseq RNA

In [10]:
print(len(hiseq_patient_set))
print(len(hiseq_patient_set & clinical_patient_set))

433
396


In [11]:
print(len(ga_patient_set))
print(len(ga_patient_set & clinical_patient_set))

263
263


In [12]:
hiseq_clinical_shared_set = hiseq_patient_set & clinical_patient_set
unique_hiseq_clinical_shared_set = hiseq_clinical_shared_set - ga_patient_set
len(unique_hiseq_clinical_shared_set)

359

In [16]:
with open('data/illumina_hiseq_patients.tsv', 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter='\t')
    spamwriter.writerow(unique_hiseq_clinical_shared_set)

In [18]:
with open('data/illumina_ga_patients.tsv', 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter='\t')
    spamwriter.writerow(ga_patient_set)

#### How many patients GA and RNA share?

In [26]:
len(hiseq_patient_set & ga_patient_set)

37

### Combine

In [23]:
combined_rna_df = trunc_ga_rna_df.copy()
for patient in unique_hiseq_clinical_shared_set:
    combined_rna_df[patient] = trunc_hiseq_rna_df[patient]

In [24]:
len(combined_rna_df.columns)

622

In [27]:
combined_rna_df.to_csv('data/trunc_combined_rna_data.tsv', sep='\t')